<a href="https://colab.research.google.com/github/cesong2/Display-Sensor/blob/Chan/comp%EC%A2%85%ED%95%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import numpy as np

In [22]:
from datetime import timedelta
import datetime
import numpy as np
from datetime import datetime
import time

In [3]:
#Load the data
telemetry_df = pd.read_csv("/content/PdM_telemetry.csv")
errors_df = pd.read_csv("/content/PdM_errors.csv")
maint_df = pd.read_csv("/content/PdM_maint.csv")
failures_df = pd.read_csv("/content/PdM_failures.csv")
machines_df = pd.read_csv("/content/PdM_machines.csv")
machines_age = pd.DataFrame(machines_df[['machineID','age']]).set_index('machineID')
machines_errors = pd.DataFrame(errors_df.groupby('machineID')['errorID'].count())
machines_maint = pd.DataFrame(maint_df.groupby('machineID')['comp'].count())
machines_failures = pd.DataFrame(failures_df.groupby('machineID')['failure'].count())

#총 몇번 교체되었는지 component별 확인

In [130]:
#maint 데이터파일 0과 1로 정리해주기
import numpy as np
comp_rep = pd.get_dummies(maint_df.set_index('datetime')).reset_index()
comp_rep.columns = ['datetime', 'machineID','comp1', 'comp2', 'comp3', 'comp4']
comp_rep = telemetry_df[['datetime', 'machineID']].merge(comp_rep,on=['datetime','machineID'],how='outer').fillna(0).sort_values(by=['machineID', 'datetime'])

In [131]:
#maint 데이터파일 시간대별로 채우기
components = ['comp1', 'comp2', 'comp3', 'comp4']
for comp in components:
    comp_rep.loc[comp_rep[comp] < 1, comp] = None
    comp_rep.loc[-comp_rep[comp].isnull(),
                 comp] = comp_rep.loc[-comp_rep[comp].isnull(), 'datetime']
    comp_rep[comp] = comp_rep[comp].fillna(method='ffill')

In [132]:
#컴포넌트별 모든 시간대 정리
comp1_t = pd.DataFrame(comp_rep, columns = ['datetime','machineID','comp1'])
comp2_t = pd.DataFrame(comp_rep, columns = ['datetime','machineID','comp2'])
comp3_t = pd.DataFrame(comp_rep, columns = ['datetime','machineID','comp3'])
comp4_t = pd.DataFrame(comp_rep, columns = ['datetime','machineID','comp4'])

In [133]:
#컴포넌트 시간대별 중복 데이터 정리 (586799데이터 中)
comp1_rep = comp1_t.drop_duplicates(['comp1']) #303데이터열
comp2_rep = comp2_t.drop_duplicates(['comp2']) #317데이터열
comp3_rep = comp3_t.drop_duplicates(['comp3']) #311데이터열
comp4_rep = comp4_t.drop_duplicates(['comp4']) #314데이터열

In [134]:
comp1_t

,datetime,machineID,comp1
275966,2014-06-01 06:00:00,1,NaN
275967,2014-07-16 06:00:00,1,NaN
275968,2014-07-31 06:00:00,1,NaN
275969,2014-12-13 06:00:00,1,2014-12-13 06:00:00
0,2015-01-01 06:00:00,1,2014-12-13 06:00:00
...,...,...,...
278330,2015-10-10 06:00:00,100,2015-10-10 06:00:00
278331,2015-10-25 06:00:00,100,2015-10-10 06:00:00
278332,2015-11-09 06:00:00,100,2015-10-10 06:00:00
278333,2015-12-09 06:00:00,100,2015-10-10 06:00:00


In [135]:
print(f'comp1은 {len(comp1_rep)}번 교체, comp2은 {len(comp2_rep)}번 교체, comp3은 {len(comp3_rep)}번 교체, comp4은 {len(comp4_rep)}번 교체,')

comp1은 303번 교체, comp2은 317번 교체, comp3은 311번 교체, comp4은 314번 교체,


#고장에 의한 교체 찾기 $1
---

In [136]:
comp1 = comp1_rep.set_index('datetime').reset_index()
comp2 = comp2_rep.set_index('datetime').reset_index()
comp3 = comp3_rep.set_index('datetime').reset_index()
comp4 = comp4_rep.set_index('datetime').reset_index()

In [137]:
#결측치 제거 전 (Nan값제거)
comp1
comp2
comp3
comp4
#결측치 제거 후 (Nan값제거)
comp1_n = comp1.dropna(axis=0) #결측치 1개 삭제
comp2_n = comp2.dropna(axis=0) #결측치 0개 삭제
comp3_n = comp3.dropna(axis=0) #결측치 1개 삭제
comp4_n = comp4.dropna(axis=0) #결측치 1개 삭제
#순서 재배치
comp1_n = comp1_n.set_index('datetime').reset_index()
comp2_n = comp2_n.set_index('datetime').reset_index()
comp3_n = comp3_n.set_index('datetime').reset_index()
comp4_n = comp4_n.set_index('datetime').reset_index()

In [138]:
df = pd.DataFrame(columns = {'failures_maint','machineID','datetime'})
df = df.reindex(columns = ['machineID','datetime','failures_maint'])
df

,machineID,datetime,failures_maint


In [139]:
n = maint_df.merge(failures_df, on = ('machineID','datetime'),how = 'inner')
for i in range(len(n)):
  if n['comp'][i] == n['failure'][i]: #겹치는애들
    df.loc[i] = [n['machineID'][i],n['datetime'][i],n['failure'][i]]
  else:#겹치지 않는애들 (제거 요망)
    pass

In [140]:
df1 = df.set_index('datetime').reset_index() #숫자 재정렬
df1

,datetime,machineID,failures_maint
0,2015-01-05 06:00:00,1,comp4
1,2015-03-06 06:00:00,1,comp1
2,2015-04-20 06:00:00,1,comp2
3,2015-06-19 06:00:00,1,comp4
4,2015-09-02 06:00:00,1,comp4
...,...,...,...
738,2015-11-29 06:00:00,99,comp3
739,2015-12-14 06:00:00,99,comp4
740,2015-02-12 06:00:00,100,comp1
741,2015-09-10 06:00:00,100,comp1


In [141]:
df_comp1 = df1[df1['failures_maint']== 'comp1'] #failures_maint별 정리
df_comp2 = df1[df1['failures_maint']== 'comp2']
df_comp3 = df1[df1['failures_maint']== 'comp3']
df_comp4 = df1[df1['failures_maint']== 'comp4']
df_com1 = df_comp1.set_index('datetime').reset_index()#숫자 재정렬
df_com2 = df_comp2.set_index('datetime').reset_index()
df_com3 = df_comp3.set_index('datetime').reset_index()
df_com4 = df_comp4.set_index('datetime').reset_index()

In [142]:
#datetime 을 이용한 교체주기 확인
fail_com = []
n = 1
for i in range(len(df_com1)-1):
  if n != range(len(df_com1)):
    str_datetime1 = df_com1["datetime"][n]
    str_datetime2 = df_com1["datetime"][i]
    currdate1 = datetime.strptime(str_datetime1, '%Y-%m-%d %H:%M:%S')
    currdate2 = datetime.strptime(str_datetime2, '%Y-%m-%d %H:%M:%S')
    new = currdate1- currdate2
    fail_com.append(new.days)
    n += 1
  else:
    break
fail_com1 = []
n1 = 1
for i in range(len(df_com2)-1):
  if n1 != range(len(df_com2)):
    str_datetime1 = df_com2["datetime"][n1]
    str_datetime2 = df_com2["datetime"][i]
    currdate1 = datetime.strptime(str_datetime1, '%Y-%m-%d %H:%M:%S')
    currdate2 = datetime.strptime(str_datetime2, '%Y-%m-%d %H:%M:%S')
    new = currdate1- currdate2
    fail_com1.append(new.days)
    n1 += 1
  else:
    break
fail_com2 = []
n2 = 1
for i in range(len(df_com3)-1):
  if n2 != range(len(df_com3)):
    str_datetime1 = df_com3["datetime"][n2]
    str_datetime2 = df_com3["datetime"][i]
    currdate1 = datetime.strptime(str_datetime1, '%Y-%m-%d %H:%M:%S')
    currdate2 = datetime.strptime(str_datetime2, '%Y-%m-%d %H:%M:%S')
    new = currdate1- currdate2
    fail_com2.append(new.days)
    n2 += 1
  else:
    break
fail_com3 = []
n3 = 1
for i in range(len(df_com4)-1):
  if n3 != range(len(df_com4)):
    str_datetime1 = df_com4["datetime"][n3]
    str_datetime2 = df_com4["datetime"][i]
    currdate1 = datetime.strptime(str_datetime1, '%Y-%m-%d %H:%M:%S')
    currdate2 = datetime.strptime(str_datetime2, '%Y-%m-%d %H:%M:%S')
    new = currdate1- currdate2
    fail_com3.append(new.days)
    n3 += 1
  else:
    break

In [143]:
#리스트 안의 음수를 none으로 바꾸기 #def로 바꾸기
fail_comp =[]
for i in range(len(fail_com)):
  if fail_com[i] > 0:
    fail_comp.append(fail_com[i])
  else:
    fail_comp.append(None)
fail_comp1 =[]
for i in range(len(fail_com1)):
  if fail_com1[i] > 0:
    fail_comp1.append(fail_com1[i])
  else:
    fail_comp1.append(None)
fail_comp2 =[]
for i in range(len(fail_com2)):
  if fail_com2[i] > 0:
    fail_comp2.append(fail_com2[i])
  else:
    fail_comp2.append(None)
fail_comp3 =[]
for i in range(len(fail_com3)):
  if fail_com3[i] > 0:
    fail_comp3.append(fail_com3[i])
  else:
    fail_comp3.append(None)

In [144]:
#마지막 한 열은 다음 계산할것이 없으니 None값 하나를 추가해줘서 index개수 맞춰주기
fail_comp.append(None)
fail_comp1.append(None) 
fail_comp2.append(None) 
fail_comp3.append(None) 

In [145]:
#기존데이터에 dataframe으로 추가해주기
df_com1['time_diff'] = fail_comp
df_com2['time_diff'] = fail_comp1
df_com3['time_diff'] = fail_comp2
df_com4['time_diff'] = fail_comp3

In [146]:
#def 로 바꾸기
li_co = []#한개씩가져오기
li1_co = []#최소 한개 이상인 데이터 머신아이디 뽑아오기
for i in range(len(df_com1)):
  if df_com1['machineID'][i] not in li_co:
    li_co.append(df_com1['machineID'][i])
  else:
    li1_co.append(df_com1['machineID'][i])
li_co1 = []#한개씩가져오기
li1_co1 = []#최소 한개 이상인 데이터 머신아이디 뽑아오기
for i in range(len(df_com1)):
  if df_com1['machineID'][i] not in li_co1:
    li_co1.append(df_com1['machineID'][i])
  else:
    li1_co1.append(df_com1['machineID'][i])
li_co2 = []#한개씩가져오기
li1_co2 = []#최소 한개 이상인 데이터 머신아이디 뽑아오기
for i in range(len(df_com1)):
  if df_com1['machineID'][i] not in li_co2:
    li_co2.append(df_com1['machineID'][i])
  else:
    li1_co2.append(df_com1['machineID'][i])
li_co3 = []#한개씩가져오기
li1_co3 = []#최소 한개 이상인 데이터 머신아이디 뽑아오기
for i in range(len(df_com1)):
  if df_com1['machineID'][i] not in li_co3:
    li_co3.append(df_com1['machineID'][i])
  else:
    li1_co3.append(df_com1['machineID'][i])
  

In [147]:
#중복처리
li1_co=set(li1_co)
li1_co1=set(li1_co1)
li1_co2=set(li1_co2)
li1_co3=set(li1_co3)

In [148]:
#한개씩 있는 머신아이디 추출
fail_c = []
for i in range(len(li_co)):
  if li_co[i] not in li1_co:
    fail_c.append(li_co[i])
  else:
    pass
fail_c1 = []
for i in range(len(li_co1)):
  if li_co1[i] not in li1_co1:
    fail_c1.append(li_co1[i])
  else:
    pass
fail_c2 = []
for i in range(len(li_co2)):
  if li_co2[i] not in li1_co2:
    fail_c2.append(li_co2[i])
  else:
    pass
fail_c3 = []
for i in range(len(li_co3)):
  if li_co3[i] not in li1_co3:
    fail_c3.append(li_co3[i])
  else:
    pass

In [149]:
#한개씩 있는 machineID의 time_diff를 nan값으로 처리
for i in range(len(fail_c)):
  df_com1.loc[df_com1['machineID']==fail_c[i],'time_diff'] = None
for i in range(len(fail_c1)):
  df_com2.loc[df_com2['machineID']==fail_c1[i],'time_diff'] = None
for i in range(len(fail_c2)):
  df_com3.loc[df_com3['machineID']==fail_c2[i],'time_diff'] = None
for i in range(len(fail_c3)):
  df_com4.loc[df_com4['machineID']==fail_c3[i],'time_diff'] = None

In [150]:
ans1 = df_com1['time_diff'].mean().round(2)
ans2 = df_com2['time_diff'].mean().round(2)
ans3= df_com3['time_diff'].mean().round(2)
ans4= df_com4['time_diff'].mean().round(2)
print(f'고장난 컴포넌트 1의 평균 교체주기 (day)는 {ans1}일입니다,   컴포넌트 2의 평균 교체주기 (day)는 {ans2}일입니다,    컴포넌트 3의 평균 교체주기 (day)는 {ans3}일입니다,    컴포넌트 4의 평균 교체주기 (day)는 {ans4}일입니다')

고장난 컴포넌트 1의 평균 교체주기 (day)는 107.43일입니다,   컴포넌트 2의 평균 교체주기 (day)는 89.36일입니다,    컴포넌트 3의 평균 교체주기 (day)는 87.53일입니다,    컴포넌트 4의 평균 교체주기 (day)는 95.39일입니다


#고장제외) 컴포넌트 교체 주기 $2

In [293]:
df_1 = pd.DataFrame(columns = {'failures_maint','machineID','datetime'})
df_1 = df_1.reindex(columns = ['machineID','datetime','failures_maint'])
df_1

,machineID,datetime,failures_maint


In [294]:
def find_diff(x,y): #x = comp1_n, y = df_com1
  list_test = [] #고장난 데이터
  list_test_positive = [] #고장 안난데이터
  list_id = []
  list_id_positive = []
  n = 0
  for i in range(len(x)):
    if x['datetime'][i] == y['datetime'][n]: #겹치는애들 = 고장난애들
      list_test.append(x['datetime'][i])
      list_id.append(x['machineID'][i])
      n+=1
    else:
      #겹치지 않는애들 = 고장안난애들
      list_test_positive.append(x['datetime'][i])
      list_id_positive.append(x['machineID'][i])
  return

In [295]:
find_diff(comp3_n,df_com3)

In [296]:
#전체 교체주기 - 고장 교체주기 = 일반 교체주기
list_test_positive = [] #고장 안난데이터
list_id_positive = []
n = 0
for i in range(len(comp1_n)):
  if comp1_n['datetime'][i] == df_com1['datetime'][n]: #겹치는애들 = 고장난애들
    n+=1
  else:
    #겹치지 않는애들 = 고장안난애들
    list_test_positive.append(comp1_n['datetime'][i])
    list_id_positive.append(comp1_n['machineID'][i])

list_test_positive2 = [] #고장 안난데이터
list_id_positive2 = []
n1 = 0
for i in range(len(comp2_n)):
  if comp2_n['datetime'][i] == df_com2['datetime'][n1]: #겹치는애들 = 고장난애들
    n1+=1
  else:
    #겹치지 않는애들 = 고장안난애들
    list_test_positive2.append(comp2_n['datetime'][i])
    list_id_positive2.append(comp2_n['machineID'][i])

list_test_positive3 = [] #고장 안난데이터
list_id_positive3 = []
n2 = 0

for i in range(len(comp3_n)):
  if comp3_n['datetime'][i] == df_com3['datetime'][n2]: #겹치는애들 = 고장난애들
    n2+=1
  else:
    #겹치지 않는애들 = 고장안난애들
    list_test_positive3.append(comp3_n['datetime'][i])
    list_id_positive3.append(comp3_n['machineID'][i])


list_test_positive4 = [] #고장 안난데이터
list_id_positive4 = []
n3 = 0
for i in range(len(comp4_n)):
  if comp4_n['datetime'][i] == df_com4['datetime'][n3]: #겹치는애들 = 고장난애들
    n3+=1
  else:
    #겹치지 않는애들 = 고장안난애들
    list_test_positive4.append(comp4_n['datetime'][i])
    list_id_positive4.append(comp4_n['machineID'][i])

In [297]:
df_test= pd.DataFrame(data=list(zip(list_test_positive,list_id_positive)),columns = ['datetime','machineID'])
df_test1= pd.DataFrame(data=list(zip(list_test_positive2,list_id_positive2)),columns = ['datetime','machineID'])
df_test2= pd.DataFrame(data=list(zip(list_test_positive3,list_id_positive3)),columns = ['datetime','machineID'])
df_test3= pd.DataFrame(data=list(zip(list_test_positive4,list_id_positive4)),columns = ['datetime','machineID'])

In [298]:
df_test['comp1'] = 'comp1'
df_test1['comp2'] = 'comp2'
df_test2['comp3'] = 'comp3'
df_test3['comp4'] = 'comp4'

In [299]:
df_test

,datetime,machineID,comp1
0,2014-12-13 06:00:00,1,comp1
1,2015-01-05 06:00:00,1,comp1
2,2015-01-20 06:00:00,1,comp1
3,2015-03-21 06:00:00,1,comp1
4,2015-06-19 06:00:00,1,comp1
...,...,...,...
282,2015-06-08 06:00:00,97,comp1
283,2015-07-23 06:00:00,97,comp1
284,2015-10-21 06:00:00,97,comp1
285,2015-03-14 06:00:00,100,comp1


In [300]:
#datetime 을 이용한 교체주기 확인
comp1_re = []
n = 1
for i in range(len(df_test)-1):
  if n != range(len(df_test)):
    str_datetime1 = df_test["datetime"][n]
    str_datetime2 = df_test["datetime"][i]
    currdate1 = datetime.strptime(str_datetime1, '%Y-%m-%d %H:%M:%S')
    currdate2 = datetime.strptime(str_datetime2, '%Y-%m-%d %H:%M:%S')
    new = currdate1- currdate2
    comp1_re.append(new.days)
    n += 1
  else:
    break
comp1_re1 = []
n = 1
for i in range(len(df_test1)-1):
  if n != range(len(df_test1)):
    str_datetime1 = df_test1["datetime"][n]
    str_datetime2 = df_test1["datetime"][i]
    currdate1 = datetime.strptime(str_datetime1, '%Y-%m-%d %H:%M:%S')
    currdate2 = datetime.strptime(str_datetime2, '%Y-%m-%d %H:%M:%S')
    new = currdate1- currdate2
    comp1_re1.append(new.days)
    n += 1
  else:
    break
comp1_re2 = []
n = 1
for i in range(len(df_test2)-1):
  if n != range(len(df_test2)):
    str_datetime1 = df_test2["datetime"][n]
    str_datetime2 = df_test2["datetime"][i]
    currdate1 = datetime.strptime(str_datetime1, '%Y-%m-%d %H:%M:%S')
    currdate2 = datetime.strptime(str_datetime2, '%Y-%m-%d %H:%M:%S')
    new = currdate1- currdate2
    comp1_re2.append(new.days)
    n += 1
  else:
    break
comp1_re3 = []
n = 1
for i in range(len(df_test3)-1):
  if n != range(len(df_test3)):
    str_datetime1 = df_test3["datetime"][n]
    str_datetime2 = df_test3["datetime"][i]
    currdate1 = datetime.strptime(str_datetime1, '%Y-%m-%d %H:%M:%S')
    currdate2 = datetime.strptime(str_datetime2, '%Y-%m-%d %H:%M:%S')
    new = currdate1- currdate2
    comp1_re3.append(new.days)
    n += 1
  else:
    break

In [301]:
# 각 데이터 별 개수
len(comp1_re),len(comp1_re1),len(comp1_re2),len(comp1_re3)

(286, 300, 301, 307)

In [302]:
#리스트 안의 음수를 none으로 바꾸기 #def로 바꾸기
comp1_a1 =[]
for i in range(len(comp1_re)):
  if comp1_re[i] > 0:
    comp1_a1.append(comp1_re[i])
  else:
    comp1_a1.append(None)
comp1_a2 =[]
for i in range(len(comp1_re1)):
  if comp1_re1[i] > 0:
    comp1_a2.append(comp1_re1[i])
  else:
    comp1_a2.append(None)
comp1_a3 =[]
for i in range(len(comp1_re2)):
  if comp1_re2[i] > 0:
    comp1_a3.append(comp1_re2[i])
  else:
    comp1_a3.append(None)
comp1_a4 =[]
for i in range(len(comp1_re3)):
  if comp1_re3[i] > 0:
    comp1_a4.append(comp1_re3[i])
  else:
    comp1_a4.append(None)


In [303]:
#마지막 한 열은 다음 계산할것이 없으니 None값 하나를 추가해줘서 index개수 맞춰주기
comp1_a1.append(None) 
comp1_a2.append(None) 
comp1_a3.append(None) 
comp1_a4.append(None) 

In [304]:
#기존데이터에 dataframe으로 추가해주기
df_test['time_diff'] = comp1_a1
df_test1['time_diff'] = comp1_a2
df_test2['time_diff'] = comp1_a3
df_test3['time_diff'] = comp1_a4

In [305]:
#def 로 바꾸기
li = []#한개씩가져오기
li1 = []#최소 한개 이상인 데이터 머신아이디 뽑아오기
for i in range(len(df_test)):
  if df_test['machineID'][i] not in li:
    li.append(df_test['machineID'][i])
  else:
    li1.append(df_test['machineID'][i])


li2 = []#한개씩가져오기
li3 = []#최소 한개 이상인 데이터 머신아이디 뽑아오기
for i in range(len(df_test1)):
  if df_test1['machineID'][i] not in li2:
    li2.append(df_test1['machineID'][i])
  else:
    li3.append(df_test1['machineID'][i])


li4 = []#한개씩가져오기
li5 = []#최소 한개 이상인 데이터 머신아이디 뽑아오기
for i in range(len(df_test2)):
  if df_test2['machineID'][i] not in li4:
    li4.append(df_test2['machineID'][i])
  else:
    li5.append(df_test2['machineID'][i])


li6 = []#한개씩가져오기
li7 = []#최소 한개 이상인 데이터 머신아이디 뽑아오기
for i in range(len(df_test3)):
  if df_test3['machineID'][i] not in li6:
    li6.append(df_test3['machineID'][i])
  else:
    li7.append(df_test3['machineID'][i])

In [306]:
#중복처리
li1=set(li1)
li3=set(li3)
li5=set(li5)
li7=set(li7)

In [307]:
#한개씩 있는 머신아이디 추출
nli1 = []
for i in range(len(li)):
  if li[i] not in li1:
    nli1.append(li[i])
  else:
    pass

nli2 = []
for i in range(len(li2)):
  if li2[i] not in li3:
    nli2.append(li2[i])
  else:
    pass

nli3 = []
for i in range(len(li4)):
  if li4[i] not in li5:
    nli3.append(li4[i])
  else:
    pass

nli4 = []
for i in range(len(li6)):
  if li6[i] not in li7:
    nli4.append(li6[i])
  else:
    pass

In [308]:
 #한개씩 있는 머신아이디 추출
nli1
nli2
nli3
nli4

[15, 34, 43, 51, 52, 60, 76, 80, 82, 86, 88, 90, 94, 97, 100]

In [309]:
#nli를 이용하여 한개씩 있는 machineID의 time_diff를 nan값으로 처리
for i in range(len(nli1)):
  df_test.loc[df_test['machineID']==nli1[i],'time_diff'] = None
for i in range(len(nli2)):
  df_test1.loc[df_test1['machineID']==nli2[i],'time_diff'] = None
for i in range(len(nli3)):
  df_test2.loc[df_test2['machineID']==nli3[i],'time_diff'] = None
for i in range(len(nli4)):
  df_test3.loc[df_test3['machineID']==nli4[i],'time_diff'] = None

In [310]:
#데이터 개수 확인
len(df_test),len(df_test1),len(df_test2),len(df_test3)

(287, 301, 302, 308)

In [311]:
df_test

,datetime,machineID,comp1,time_diff
0,2014-12-13 06:00:00,1,comp1,23.0
1,2015-01-05 06:00:00,1,comp1,15.0
2,2015-01-20 06:00:00,1,comp1,60.0
3,2015-03-21 06:00:00,1,comp1,90.0
4,2015-06-19 06:00:00,1,comp1,30.0
...,...,...,...,...
282,2015-06-08 06:00:00,97,comp1,45.0
283,2015-07-23 06:00:00,97,comp1,90.0
284,2015-10-21 06:00:00,97,comp1,NaN
285,2015-03-14 06:00:00,100,comp1,195.0


In [313]:
#day기준
df_test['time_diff'].describe()
df_test1['time_diff'].describe()
df_test2['time_diff'].describe()
df_test3['time_diff'].describe()
df_test_t=df_test['time_diff'].mean().round(3)
df_test1_t=df_test1['time_diff'].mean().round(3)
df_test2_t=df_test2['time_diff'].mean().round(3)
df_test3_t=df_test3['time_diff'].mean().round(3)
print(f'컴포넌트 일반적인 교체 주기 종합: 1의 평균 교체주기 (day)는 {df_test_t}일입니다,   컴포넌트 2의 평균 교체주기 (day)는 {df_test1_t}일입니다,    컴포넌트 3의 평균 교체주기 (day)는 {df_test2_t}일입니다,    컴포넌트 4의 평균 교체주기 (day)는 {df_test3_t}일입니다')

컴포넌트 일반적인 교체 주기 종합: 1의 평균 교체주기 (day)는 77.766일입니다,   컴포넌트 2의 평균 교체주기 (day)는 66.969일입니다,    컴포넌트 3의 평균 교체주기 (day)는 78.018일입니다,    컴포넌트 4의 평균 교체주기 (day)는 69.81일입니다


#고장+일반교체) 교체주기 확인 $3

---



In [261]:
#datetime 을 이용한 교체주기 확인
comp1_re = []
n = 1
for i in range(len(comp1_n)-1):
  if n != range(len(comp1_n)):
    str_datetime1 = comp1_n["comp1"][n]
    str_datetime2 = comp1_n["comp1"][i]
    currdate1 = datetime.strptime(str_datetime1, '%Y-%m-%d %H:%M:%S')
    currdate2 = datetime.strptime(str_datetime2, '%Y-%m-%d %H:%M:%S')
    new = currdate1- currdate2
    comp1_re.append(new.days)
    n += 1
  else:
    break
comp2_re = []
n1 = 1
for i in range(len(comp2_n)-1):
  if n1 != range(len(comp2_n)):
    str_datetime1 = comp2_n["comp2"][n1]
    str_datetime2 = comp2_n["comp2"][i]
    currdate1 = datetime.strptime(str_datetime1, '%Y-%m-%d %H:%M:%S')
    currdate2 = datetime.strptime(str_datetime2, '%Y-%m-%d %H:%M:%S')
    new = currdate1- currdate2
    comp2_re.append(new.days)
    n1 += 1
  else:
    break
comp3_re = []
n2 = 1
for i in range(len(comp3_n)-1):
  if n2 != range(len(comp3_n)):
    str_datetime1 = comp3_n["comp3"][n2]
    str_datetime2 = comp3_n["comp3"][i]
    currdate1 = datetime.strptime(str_datetime1, '%Y-%m-%d %H:%M:%S')
    currdate2 = datetime.strptime(str_datetime2, '%Y-%m-%d %H:%M:%S')
    new = currdate1- currdate2
    comp3_re.append(new.days)
    n2 += 1
  else:
    break
comp4_re = []
n3 = 1
for i in range(len(comp4_n)-1):
  if n3 != range(len(comp4_n)):
    str_datetime1 = comp4_n["comp4"][n3]
    str_datetime2 = comp4_n["comp4"][i]
    currdate1 = datetime.strptime(str_datetime1, '%Y-%m-%d %H:%M:%S')
    currdate2 = datetime.strptime(str_datetime2, '%Y-%m-%d %H:%M:%S')
    new = currdate1- currdate2
    comp4_re.append(new.days)
    n3 += 1
  else:
    break

In [262]:
# 음수인 데이터는 machineID 가 바뀌면서 생기는 데이터로 사실상 의미가없음.
len(comp1_re),len(comp2_re),len(comp3_re),len(comp4_re)

(301, 316, 309, 312)

In [263]:
#리스트 안의 음수를 none으로 바꾸기 #def로 바꾸기
comp1_re1 =[]
for i in range(len(comp1_re)):
  if comp1_re[i] > 0:
    comp1_re1.append(comp1_re[i])
  else:
    comp1_re1.append(None)
  
comp2_re1 =[]
for i in range(len(comp2_re)):
  if comp2_re[i] > 0:
    comp2_re1.append(comp2_re[i])
  else:
    comp2_re1.append(None)
  
comp3_re1 =[]
for i in range(len(comp3_re)):
  if comp3_re[i] > 0:
    comp3_re1.append(comp3_re[i])
  else:
    comp3_re1.append(None)
  
comp4_re1 =[]
for i in range(len(comp4_re)):
  if comp4_re[i] > 0:
    comp4_re1.append(comp4_re[i])
  else:
    comp4_re1.append(None)

In [264]:
#마지막 한 열은 다음 계산할것이 없으니 None값 하나를 추가해줘서 index개수 맞춰주기
comp1_re1.append(None) 
comp2_re1.append(None) 
comp3_re1.append(None) 
comp4_re1.append(None) 

In [265]:
#기존데이터에 dataframe으로 추가해주기
comp1_n['time_diff'] = comp1_re1
comp2_n['time_diff'] = comp2_re1
comp3_n['time_diff'] = comp3_re1
comp4_n['time_diff'] = comp4_re1

In [266]:
#def 로 바꾸기
li = []#한개씩가져오기
li1 = []#최소 한개 이상인 데이터 머신아이디 뽑아오기
for i in range(len(comp1_n)):
  if comp1_n['machineID'][i] not in li:
    li.append(comp1_n['machineID'][i])
  else:
    li1.append(comp1_n['machineID'][i])


li2 = []#한개씩가져오기
li3 = []#최소 한개 이상인 데이터 머신아이디 뽑아오기
for i in range(len(comp2_n)):
  if comp2_n['machineID'][i] not in li2:
    li2.append(comp2_n['machineID'][i])
  else:
    li3.append(comp2_n['machineID'][i])


li4 = []#한개씩가져오기
li5 = []#최소 한개 이상인 데이터 머신아이디 뽑아오기
for i in range(len(comp3_n)):
  if comp3_n['machineID'][i] not in li4:
    li4.append(comp3_n['machineID'][i])
  else:
    li5.append(comp3_n['machineID'][i])


li6 = []#한개씩가져오기
li7 = []#최소 한개 이상인 데이터 머신아이디 뽑아오기
for i in range(len(comp4_n)):
  if comp4_n['machineID'][i] not in li6:
    li6.append(comp4_n['machineID'][i])
  else:
    li7.append(comp4_n['machineID'][i])

In [267]:
#중복처리
li1=set(li1)
li3=set(li3)
li5=set(li5)
li7=set(li7)

In [268]:
#한개씩 있는 머신아이디 추출
nli1 = []
for i in range(len(li)):
  if li[i] not in li1:
    nli1.append(li[i])
  else:
    pass

nli2 = []
for i in range(len(li2)):
  if li2[i] not in li3:
    nli2.append(li2[i])
  else:
    pass

nli3 = []
for i in range(len(li4)):
  if li4[i] not in li5:
    nli3.append(li4[i])
  else:
    pass

nli4 = []
for i in range(len(li6)):
  if li6[i] not in li7:
    nli4.append(li6[i])
  else:
    pass

In [269]:
 #한개씩 있는 머신아이디 추출
nli1
nli2
nli3
nli4

[15, 34, 43, 51, 52, 60, 76, 80, 82, 86, 88, 90, 94, 97, 100]

In [270]:
#nli를 이용하여 한개씩 있는 machineID의 time_diff를 nan값으로 처리
for i in range(len(nli1)):
  comp1_n.loc[comp1_n['machineID']==nli1[i],'time_diff'] = None
for i in range(len(nli2)):
  comp2_n.loc[comp2_n['machineID']==nli2[i],'time_diff'] = None
for i in range(len(nli3)):
  comp3_n.loc[comp3_n['machineID']==nli3[i],'time_diff'] = None
for i in range(len(nli4)):
  comp4_n.loc[comp4_n['machineID']==nli4[i],'time_diff'] = None

In [271]:
#데이터 개수 확인
len(comp1_n),len(comp2_n),len(comp3_n),len(comp4_n)

(302, 317, 310, 313)

In [272]:
#day기준
comp1_n['time_diff'].describe()
comp2_n['time_diff'].describe()
comp3_n['time_diff'].describe()
comp4_n['time_diff'].describe()
comp_mean1=comp1_n['time_diff'].mean().round(3)
comp_mean2=comp2_n['time_diff'].mean().round(3)
comp_mean3=comp3_n['time_diff'].mean().round(3)
comp_mean4=comp4_n['time_diff'].mean().round(3)
print(f'컴포넌트 1의 평균 교체주기 (day)는 {comp_mean1}일입니다,   컴포넌트 2의 평균 교체주기 (day)는 {comp_mean2}일입니다,    컴포넌트 3의 평균 교체주기 (day)는 {comp_mean3}일입니다,    컴포넌트 4의 평균 교체주기 (day)는 {comp_mean4}일입니다')

컴포넌트 1의 평균 교체주기 (day)는 73.655일입니다,   컴포넌트 2의 평균 교체주기 (day)는 63.177일입니다,    컴포넌트 3의 평균 교체주기 (day)는 76.194일입니다,    컴포넌트 4의 평균 교체주기 (day)는 69.667일입니다


#전체적인 컴포넌트 데이터 종합본

In [316]:
print(f'고장난 컴포넌트 1의 평균 교체주기 (day)는 {ans1}일입니다,   컴포넌트 2의 평균 교체주기 (day)는 {ans2}일입니다,    컴포넌트 3의 평균 교체주기 (day)는 {ans3}일입니다,    컴포넌트 4의 평균 교체주기 (day)는 {ans4}일입니다')

고장난 컴포넌트 1의 평균 교체주기 (day)는 107.43일입니다,   컴포넌트 2의 평균 교체주기 (day)는 89.36일입니다,    컴포넌트 3의 평균 교체주기 (day)는 87.53일입니다,    컴포넌트 4의 평균 교체주기 (day)는 95.39일입니다


In [314]:
print(f'컴포넌트 일반적인 교체 주기 종합: 1의 평균 교체주기 (day)는 {df_test_t}일입니다,   컴포넌트 2의 평균 교체주기 (day)는 {df_test1_t}일입니다,    컴포넌트 3의 평균 교체주기 (day)는 {df_test2_t}일입니다,    컴포넌트 4의 평균 교체주기 (day)는 {df_test3_t}일입니다')

컴포넌트 일반적인 교체 주기 종합: 1의 평균 교체주기 (day)는 77.766일입니다,   컴포넌트 2의 평균 교체주기 (day)는 66.969일입니다,    컴포넌트 3의 평균 교체주기 (day)는 78.018일입니다,    컴포넌트 4의 평균 교체주기 (day)는 69.81일입니다


In [315]:
print(f'컴포넌트 1의 평균 교체주기 (day)는 {comp_mean1}일입니다,   컴포넌트 2의 평균 교체주기 (day)는 {comp_mean2}일입니다,    컴포넌트 3의 평균 교체주기 (day)는 {comp_mean3}일입니다,    컴포넌트 4의 평균 교체주기 (day)는 {comp_mean4}일입니다')

컴포넌트 1의 평균 교체주기 (day)는 73.655일입니다,   컴포넌트 2의 평균 교체주기 (day)는 63.177일입니다,    컴포넌트 3의 평균 교체주기 (day)는 76.194일입니다,    컴포넌트 4의 평균 교체주기 (day)는 69.667일입니다
